In [1]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import confusion_matrix

In [2]:
data = pd.read_csv("spam.csv", encoding = "latin-1")

In [3]:
data = data[['v1', 'v2']]

In [4]:
data = data.rename(columns = {'v1': 'label', 'v2': 'text'})

In [5]:
lemmatizer = WordNetLemmatizer()

In [6]:
stopwords = set(stopwords.words('english'))

In [7]:
def review_messages(msg):
    # converting messages to lowercase
    msg = msg.lower()
    return msg

In [8]:
def alternative_review_messages(msg):
    # converting messages to lowercase
    msg = msg.lower()

    # uses a lemmatizer (wnpos is the parts of speech tag)
    # unfortunately wordnet and nltk uses a different set of terminology for pos tags
    # first, we must translate the nltk pos to wordnet
    nltk_pos = [tag[1] for tag in pos_tag(word_tokenize(msg))]
    msg = [tag[0] for tag in pos_tag(word_tokenize(msg))]
    wnpos = ['a' if tag[0] == 'J' else tag[0].lower() if tag[0] in ['N', 'R', 'V'] else 'n' for tag in nltk_pos]
    msg = " ".join([lemmatizer.lemmatize(word, wnpos[i]) for i, word in enumerate(msg)])

    # removing stopwords 
    msg = [word for word in msg.split() if word not in stopwords]
    msg = " ".join(msg)
    return msg

In [9]:
# Processing text messages
data['text'] = data['text'].apply(alternative_review_messages)

In [10]:
# train test split 
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size = 0.1, random_state = 1)

In [11]:
# training vectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [12]:
# training the classifier 
svm = svm.SVC(C=1000)
svm.fit(X_train, y_train)

/home/kumar/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [13]:
# testing against testing set 
X_test = vectorizer.transform(X_test)
y_pred = svm.predict(X_test) 
print(confusion_matrix(y_test, y_pred))

[[490   0]
 [ 10  58]]


In [14]:
svm.score(X_test,y_test)

0.982078853046595

In [21]:
# test against new messages 
def pred(msg):
    msg = alternative_review_messages(msg)
    msg = vectorizer.transform([msg])
    prediction = svm.predict(msg)
    return prediction[0]

In [22]:
test = 'Our records indicate your Pension is under performing to see higher growth and up to 25% cash release reply PENSION for a free review. To opt out reply STOP'
my_prediction = pred(test)
my_prediction

'spam'